In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#CLASSIFIERS FOR TRAINING
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

In [ ]:
data = pd.read_csv("../input/fetal-health-classification/fetal_health.csv") 
print("♦ LOOK AND CHECK DATA:")
print(data.head())
print()
print("♦ DATASET LENGHT = ", len(data))

In [ ]:
cols = data.columns
print(cols)

ANALYZE DATA

One of the most important things is to understand data which you work with. Here we will use some well-known methods for easier understanding of our data.

For all dataset columns we will find some statistical information, like: Mean, Median, Mode, Standard Deviation and Correlation using Pandas functions.

In [ ]:
mean = data.mean(axis=0)
print(mean)

median = data.median(axis=0)
print(median)

mode = data.mode(axis=0)
print(mode)

std = data.std(axis=0)
print(std)

print(data.corrwith(data["fetal_health"]))

NORMALIZE DATA

It is necessary to normalize data before making some classification tasks. Here i will normalize it in a range from 0 to 1.

In [ ]:
#STEP-1
scaler = MinMaxScaler(feature_range=(0, 1))
#STEP-2
temp = data["fetal_health"]
#STEP-3
norm_data = scaler.fit_transform(data)
#STEP-4
data = pd.DataFrame(data=norm_data, columns=cols)
#STEP-5
data["fetal_health"] = temp
#STEP-6
print(data.head())

SPLIT DATASET INTO TRAIN AND TEST DATA

So, here i will divide the dataset into two parts (for model training and validation); 70%:30% respectively

In [ ]:

rng = RandomState()

#STEP-2
train = data.sample(frac=0.7, random_state=rng)
val = data.loc[~data.index.isin(train.index)]

#STEP-3
train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)

#STEP-4
print("♦ TRAIN SET:")
print(train.head())
print()
print("♦ VALIDATION SET:")
print(val.head())

In [ ]:
#STEP-1
x_columns = cols[:-1]
y_column = cols[-1]

#STEP-2
x_raw_train = train[x_columns]
y_raw_train = train[y_column]

#STEP-3
X_train = x_raw_train.copy()
Y_train = y_raw_train.copy()

#STEP-4
print("♦ X_TRAIN: ")
print(X_train.head())
print()
print("♦ Y_TRAIN: ")
print(Y_train.head())
print()

#STEP-5
x_raw_val = val[x_columns]
y_raw_val = val[y_column]

#STEP-6
X_val = x_raw_val.copy()
Y_val = y_raw_val.copy()

#STEP-7
print("♦ X_VAL: ")
print(X_val.head())
print()
print("♦ Y_VAL: ")
print(Y_val.head())

Next, we are going to use classifiers from the sklearn library on our dataset

Then we will train, validate, save and load models

In [ ]:
#STEP-1
all_classifers = [
    KNeighborsClassifier(3),
    SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    LogisticRegression()
]

#STEP-2
all_acc = {}

#STEP-3
for classifier in all_classifers:
    #STEP-4
    model = classifier
    model.fit(X_train, Y_train)
    #STEP-5
    model_pred = model.predict(X_val)
    model_acc = accuracy_score(Y_val, model_pred)
    #STEP-6
    classfier_name = classifier.__class__.__name__
    #STEP-7
    all_acc[classfier_name] = model_acc
    #STEP-8
    filename = classfier_name+'_model.pickle'
    pickle.dump(model, open(filename, 'wb'))  
    #STEP-9
    loaded_model = pickle.load(open(filename, 'rb'))
    result = loaded_model.score(X_val, Y_val)     
    #STEP-10
    print("♦ {:<30} = {:<12} {:>25} = {:>12}".format(classfier_name, model_acc, 'loaded pickle model', result))

We are going to plot and visualize the output from the selected classifiers to determine which is the most appropriate

In [ ]:
#STEP-1
all_acc = dict(sorted(all_acc.items(), key=lambda item: item[1], reverse=True))

#STEP-2
keys = all_acc.keys()
values = all_acc.values()

#STEP-3
plt.figure(figsize=(10,5))
plt.title('ACCURCY OF CLASSIFIERS')
plt.xlabel('classifiers')
plt.ylabel('accuracy')
plt.bar(keys, values, color="g")

#STEP-4
plt.xticks(rotation=90)

#STEP-5
plt.grid()
plt.show()

CONCLUSION

From our plot, we can say that the GradientBoostingClassifier is the most appropriate classifier for this dataset.

Thank you for following through